# 데이터 수집 Level4

- 스타벅스의 지점 정보를 모두 뽑아보자!

- selenium 패키지를 활용하여 데이터 수집을 진행
    - https://www.selenium.dev/
    - https://www.selenium.dev/selenium/docs/api/py/index.html (api 사용법)

- 세팅
    - selenium 설치 
    - webdriver s/w 다운로드 (대부분 브라우저 밴더가 제공)
        - https://chromedriver.chromium.org/downloads
            - 주의사항 : PC에 설치된 Chrome의 버전과 일치된 버전을 다운로드
            - https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/115.0.5790.170/win64/chromedriver-win64.zip

In [1]:
# 셀레니움 모듈 가져오기

import selenium
selenium.__version__

'4.11.2'

In [2]:
from selenium import webdriver as wd
# 시간 -> 명시적으로 코드 진행을 지연시키겠다 (sleep(5))
import time

In [3]:
# 드라이버 로드
driver = wd.Chrome()

In [29]:
# 접속
target_site= 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(target_site)

In [5]:
# 탐색시 사용할 도구를 지정
# css selector, xpath, ID, class, ...
from selenium.webdriver.common.by import By

# 지역 中 서울
# div.loca_step1_cont > ul > li:nth-child(1) > a 
# 1 ~ 17 까지 숫자가 바뀌면 지역이 바뀐다.
css_selector_str = 'div.loca_step1_cont > ul > li:nth-child(1) > a'
# 요소찾기가 3.x - 4.x로 변경될 때 함수가 바뀜
sido = driver.find_element(By.CSS_SELECTOR, css_selector_str)
sido # 정상 입력 확인

<selenium.webdriver.remote.webelement.WebElement (session="959c0a469b0a13955e0861a36dd57907", element="3CAAA5E7607B94B669FFABF85F5F3433_element_20")>

In [31]:
# 서울을 클릭 > 구군이 등장
sido.click()
# 잠시 대기 -> 암묵적 대기, 명시적 대기 -> API로 제공
time.sleep(1*3) # 3초 대기

In [33]:
# 서울에 있는 모든 지점 정보?
# 군구 페이지에서 타겟팅 지정-마포구-클릭 => 지점 목록 리스트가 나타난다.
gun_gu_css_selector_str = '.gugun_arae_box > li:nth-child(1) > a'
# 요소찾기가 3.x - 4.x로 변경될 때 함수가 바뀜
gungu = driver.find_element(By.CSS_SELECTOR, gun_gu_css_selector_str)
gungu # 정상 입력 확인

<selenium.webdriver.remote.webelement.WebElement (session="b9389639ca2154fd18eff6dab721dc43", element="1A5308FF40DE3AEE59C1E285769E600D_element_22")>

In [34]:
# 마포구를 클릭 > 마포구 점포 목록 등장
gungu.click()
# 잠시 대기 -> 암묵적 대기, 명시적 대기 -> API로 제공
time.sleep(1*3) # 3초 대기

In [36]:
# 세부 정보
info_css_selector = '.quickSearchResultBoxSidoGugun > li' # 서울시 -> 전체 -> 전체 지점 정보 데이터 (601개)
stores = driver.find_elements(By.CSS_SELECTOR, info_css_selector)
print(len(stores))

601


In [41]:
# 서울시 전체 점포 추출
# 필요한 속성 추출
'''
<li class="quickResultLstCon" 
style="background:#fff" data-lat="37.501087" 
data-long="127.043069" data-index="0" 
data-name="역삼아레나빌딩" data-code="3762" 
data-storecd="1509" data-hlytag="null">	
    <strong data-store="1509" data-yn="N" 
    data-name="역삼아레나빌딩" data-my_siren_order_store_yn="N">역삼아레나빌딩  </strong>	
    <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br>1522-3232</p>	
    <i class="pin_general">리저브 매장 2번</i>
</li>
'''
# store => li 1개 데이터
for store in stores :
    # p는 store 하위에서 탐색
    p = store.find_element(By.CSS_SELECTOR, 'p') # 요소명
    # print(p.text.strip().split('\n'))
    ps = p.text.strip().split('\n') # 주소와 전화번호가 줄바꿈으로 표현됨 => 조치 필요
    # i class : 해당 매장이 reserve 매장인가? DT 매장인가? => 특징 정보 포힘
    i = store.find_element(By.CLASS_NAME, 'pin_general')
    # print(i.text.strip()) 
    data = {
    'lat'    : store.get_attribute('data-lat'),
    'long'   : store.get_attribute('data-long'),
    'name'   : store.get_attribute('data-name'),
    'code'   : store.get_attribute('data-code'),
    'stored' : store.get_attribute('data-stored'),
    'addr'   : ps[0],
    'tel'    : ps[1],
    'level'   : i.text.strip(), 
    }
    print(data)
    break

{'lat': '37.501087', 'long': '127.043069', 'name': '역삼아레나빌딩', 'code': '3762', 'stored': None, 'addr': '서울특별시 강남구 언주로 425 (역삼동)', 'tel': '1522-3232', 'level': '리저브 매장 2번'}


In [42]:
# 601개를 모으는 문장
infos = list()

for store in stores :
    p = store.find_element(By.CSS_SELECTOR, 'p')
    print(p)
    ps = p.text.strip().split('\n') 
    print(ps)
    i = store.find_element(By.CLASS_NAME, 'pin_general')
    data = {
    'lat'    : store.get_attribute('data-lat'),
    'long'   : store.get_attribute('data-long'),
    'name'   : store.get_attribute('data-name'),
    'code'   : store.get_attribute('data-code'),
    'stored' : store.get_attribute('data-stored'),
    'addr'   : ps[0],
    'tel'    : ps[1],
    'level'   : i.text.strip(), 
    }
    #print(data)
    infos.append(data)
    #break

len(infos)
# 화면에 3개씩만 보임. 그 이상 못내려가는거 아닐까? 라는 추측

<selenium.webdriver.remote.webelement.WebElement (session="b9389639ca2154fd18eff6dab721dc43", element="1A5308FF40DE3AEE59C1E285769E600D_element_628")>
['서울특별시 강남구 언주로 425 (역삼동)', '1522-3232']
<selenium.webdriver.remote.webelement.WebElement (session="b9389639ca2154fd18eff6dab721dc43", element="1A5308FF40DE3AEE59C1E285769E600D_element_630")>
['서울특별시 강남구 강남대로 538 (논현동)', '1522-3232']
<selenium.webdriver.remote.webelement.WebElement (session="b9389639ca2154fd18eff6dab721dc43", element="1A5308FF40DE3AEE59C1E285769E600D_element_632")>
['서울특별시 강남구 강남대로 584 (논현동)', '1522-3232']
<selenium.webdriver.remote.webelement.WebElement (session="b9389639ca2154fd18eff6dab721dc43", element="1A5308FF40DE3AEE59C1E285769E600D_element_634")>
['']


IndexError: list index out of range

In [44]:
'''
    문제점
    화면에 보이는 3개만 주소와 전화번호, 지점 특성이 세팅되어 있다.
        - 지점 정보를 1개 추출하고 
        - 강제 스크롤 (예상은 100px) 수행하여 다음 데이터 올려주고
            - 100px는 css를 분석해서 확인 (?)
            - 라인 1개 점검 필요함 (우선 1을 더 넣어주지)
            - 방식, selenium에서 javascript를 강제 발동시켜야함
    다시 데이터 추출(반복)하는 과정을 거쳐야할듯
'''
# 스크립트 구동
# scrollby(x, y) : 우리는 수직으로 움직이니까 y만큼 이동
for n in range(100) :
    driver.execute_script('''
        // 자바스크립트 구동 => 순수 자바스크립트만 가능
        // y축 방향으로 100px 이동
        // document.querySelector('css셀렉터').scrollBy(0, 100)
        document.querySelector('#mCSB_3').scrollBy(0, 100+1) // 데이터 여백 100px, 경계선 1px
    ''')
    time.sleep(1)

# ok 여기까지 이해, 그럼 스크롤을 작동시키는 css요소는 무엇인가?
# 보통 이런건 div 태그일 가능성이 높음
# 캡처 사진 참고
# 확인 완료! 정상적으로 이동됨

KeyboardInterrupt: 

In [46]:
# 601개를 모으는 문장
infos = list()

for store in stores[:6] :
    p = store.find_element(By.CSS_SELECTOR, 'p')
    print(p)
    ps = p.text.strip().split('\n') 
    print(ps)
    #i = store.find_element(By.CLASS_NAME, 'pin_general') 지점별로 class가 달라서 사용 불가함!
    i = store.find_element(By.TAG_NAME, 'i')

    data = {
    'lat'    : store.get_attribute('data-lat'),
    'long'   : store.get_attribute('data-long'),
    'name'   : store.get_attribute('data-name'),
    'code'   : store.get_attribute('data-code'),
    'stored' : store.get_attribute('data-stored'),
    'addr'   : ps[0],
    'tel'    : ps[1],
    'level'   : i.text.strip(), 
    }
    #print(data)
    infos.append(data)
    # 하위에 새로운 정보 1개 로드
    driver.execute_script('''
        document.querySelector('#mCSB_3').scrollBy(0, 100+1) // 데이터 여백 100px, 경계선 1px
    ''')
    time.sleep(1*0.2)
    #break

len(infos)

# 리저브 색이 다르네! 오류 발생 (다른건 다 초록색, 일부 검은색) 코드 수정

<selenium.webdriver.remote.webelement.WebElement (session="b9389639ca2154fd18eff6dab721dc43", element="1A5308FF40DE3AEE59C1E285769E600D_element_628")>
['서울특별시 강남구 언주로 425 (역삼동)', '1522-3232']
<selenium.webdriver.remote.webelement.WebElement (session="b9389639ca2154fd18eff6dab721dc43", element="1A5308FF40DE3AEE59C1E285769E600D_element_630")>
['서울특별시 강남구 강남대로 538 (논현동)', '1522-3232']
<selenium.webdriver.remote.webelement.WebElement (session="b9389639ca2154fd18eff6dab721dc43", element="1A5308FF40DE3AEE59C1E285769E600D_element_632")>
['서울특별시 강남구 강남대로 584 (논현동)', '1522-3232']
<selenium.webdriver.remote.webelement.WebElement (session="b9389639ca2154fd18eff6dab721dc43", element="1A5308FF40DE3AEE59C1E285769E600D_element_634")>
['서울특별시 강남구 테헤란로 125 (역삼동)', '1522-3232']
<selenium.webdriver.remote.webelement.WebElement (session="b9389639ca2154fd18eff6dab721dc43", element="1A5308FF40DE3AEE59C1E285769E600D_element_644")>
['서울특별시 강남구 남부순환로 2947 (대치동)', '1522-3232']
<selenium.webdriver.remote.webelem

6

In [ ]:
# 17개 지역 case별로 하부에서 지점 정보 총 획득 완료
# 한개씩은 ok, 각 지역 모두 모으려면 들어갔다가 나오는 과정들 필요함!
# 601개를 모으는 문장 (최종)
infos = list()

for store in stores[:6] :
    p = store.find_element(By.CSS_SELECTOR, 'p')
    #print(p)
    ps = p.text.strip().split('\n') 
    #print(ps)
    #i = store.find_element(By.CLASS_NAME, 'pin_general') 지점별로 class가 달라서 사용 불가함!
    i = store.find_element(By.TAG_NAME, 'i')

    data = {
    'lat'    : store.get_attribute('data-lat'),
    'long'   : store.get_attribute('data-long'),
    'name'   : store.get_attribute('data-name'),
    'code'   : store.get_attribute('data-code'),
    'stored' : store.get_attribute('data-stored'),
    'addr'   : ps[0],
    'tel'    : ps[1],
    'level'   : i.text.strip(), 
    }
    #print(data)
    infos.append(data)
    # 하위에 새로운 정보 1개 로드
    driver.execute_script('''
        document.querySelector('#mCSB_3').scrollBy(0, 100+1) // 데이터 여백 100px, 경계선 1px
    ''')
    time.sleep(1*0.2)
    #break

len(infos)

In [ ]:
# 지역 검색 누르면 지역으로 다시 돌아감

In [6]:
# 전체 데이터 수집을 위한 로테이션 시뮬레이션 <최종>
import selenium
from selenium import webdriver as wd
import time
from selenium.webdriver.common.by import By

# 1. 사이트 접속
driver = wd.Chrome()
target_site = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(target_site)

# 2. 17번 반복    
MAX_SIDO_NUM = 17
for n in range(1, MAX_SIDO_NUM):
    # 지역 검색 클릭 -> 시도 클릭 -> 전체 클릭 -> 저점정보획득(시뮬레이션상 생략)

    # 2-1. 지역 검색 클릭
    area_search_btn = driver.find_element(By.CSS_SELECTOR, '.loca_search a')
    area_search_btn.click()
    time.sleep(1*0.5)

    # 2-2. 시도 클릭
    sido_btn = driver.find_element(By.CSS_SELECTOR, f'div.loca_step1_cont > ul > li:nth-child({n}) > a')
    sido_btn.click()
    time.sleep(1)

    # 2-3. 전체 클릭
    all_btn = driver.find_element(By.CSS_SELECTOR, '.set_gugun_cd_btn')
    all_btn.click()
    time.sleep(1*7)

    # 2-4. 저점정보획득(시뮬레이션상 생략)
    pass

In [ ]:
# 최종 마무리 작업
# 웹드라이버 반드시 종료, 제거 => 메모리 누적이 안됨
driver.close()
driver.quit()

# s/w 레벨로 자동화 수행
import sys
sys.exit(0)